# MALWARE detection - 0321_SupML_OronzoComi

### Objectivo es crear un modelo que pueda predecir cual ordenador van a ser infectado por un malware.
### la variable TARGET es la siguiente: 'HasDetections':
- valor 1 = infectado
- valor 0 = no infectado

### He creado dos ficheros Excel para tener cuenta de como he manejado las variables:
- describe_columna_categoricas
- describe_columna_numericas
### He creado un fichero my_function.py y lo he importado. Ese fichero contiene las funciones que he utilizado en el notebook:
- rellena_con_valor - para rellenar los NaN de las variables numericas
- setOthers - para rellenar los NaN de las variables categoricas
- OHE - para trasformar en numerica las variables categoricas

## Importación de librerías

In [ ]:
import numpy as np
import pandas as pd

# visualizacion de datos
import matplotlib.pyplot as plt
import seaborn as sns

# para definir estilo
plt.style.use('ggplot')

# define numero max de filas que quiero que me pinte quando ejecuto
pd.set_option('display.max_rows',100) 

# para el preparado del dato
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz

# para pintar arboles de decision
import graphviz
# mis funciones en el fichero my_function.py
import my_function as mf

# Input data files are available in the read-only "../input/" directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Importación de datos 

In [ ]:
%%time
df = pd.read_csv('../input/malware-dataset/sample_mmp.csv', sep=",")

# Data Analisis
#### DataFrame Shape: rows 500000 - columns 84
#### tipo de datos: float64(36) - int64(18) - object(30) - no hay campo fechas

In [ ]:
df.info()
# DataFrame de 500'000 filas y 84 columnas
# las columnas tienen datos de tipo float64(36) - int64(18) - object(30) (no hay fechas)
# hay varios nulos

## MachineIdentifier - como index de las filas

In [ ]:
# me quedo con el MachineIdentifier como identificador unico de fila
df.set_index('MachineIdentifier',inplace=True)

## Unnamed: 0 - eliminada

In [ ]:
# borro la columna 'Unamed: 0' porque no sirve (ha sido creada durante el data import)
df.drop('Unnamed: 0', axis="columns", inplace = True)

<p style="font-size:4em; color:LawnGreen;">DATA UNDERSTANDING</p>

In [ ]:
# miro si alguno atributos tienen un solo valor
df.nunique() == 1
# PuaMode y Census_IsWIMBootEnabled se pueden eliminar porque tienen un solo valor

In [ ]:
# miro cual son las variables CATEGORICAS
df.describe(exclude=np.number).T

In [ ]:
# miro cual son las variables NUMERICAS
df.describe(include=np.number).T

# Census_IsWIMBootEnabled no tiene varianza, un solo valor 0 (eliminar)

## elimino Census_IsWIMBootEnabled

In [ ]:
# borro la columna: Census_IsWIMBootEnabled -> tiene un solo valor 0 + NULOS
df.drop('Census_IsWIMBootEnabled', axis="columns", inplace = True)


## elimino PuaMode

In [ ]:
# borro la columna: PuaMode -> tiene un solo valor ON + NULOS
df.drop('PuaMode', axis="columns", inplace = True)

## elimino -  IsBeta

In [ ]:
# borro la variable IsBeta que no tiene varianza
df.drop('IsBeta', axis="columns", inplace = True)

## creo el fichero excel para las variables CATEGORICAS

In [ ]:
# TO_EXCEL
#df_describe_categoricas = df.describe(exclude=np.number).T
#df_describe_categoricas
#df_describe_categoricas.to_excel('describe_columna_categoricas.xlsx')

In [ ]:
# aqui creo lista de las CATEGORICAS (28)
lista_columnas_categoricas = df.describe(exclude=np.number).columns.to_list()

## creo el fichero excel para las variables NUMERICAS

In [ ]:
# TO_EXCEL
#df_describe_numericas = df.describe(include=np.number).T
#df_describe_numericas
#df_describe_numericas.to_excel('describe_columna_numericas.xlsx')

In [ ]:
# aqui creo lista de las NUMERICAS (52)
lista_columnas_numericas = df.describe(include=np.number).columns.to_list()

<p style="font-size:2em; color:LawnGreen;">TARGET</p>

In [ ]:
# esta columna es mi target
# hay que encontrar un modelo que predice exactamente lo que se verifica en esta columna
# dtype: int64
target = ['HasDetections']

In [ ]:
sns.countplot(data=df, x='HasDetections')
# la distribucion de la variable target es muy balanceada

In [ ]:
# la distribucion de la variable target es muy balanceada
df[target].value_counts()

In [ ]:
# no hay valores NULOS
df[target].count()

In [ ]:
# número total de registros con 1 sobre el total, es decir, la ratio de prevalencia de la clase positiva (49.9%)
# TASA NATURAL DE APRENDIZAJE
# un 50% de los pc se infectan
df[target].mean()

<p style="font-size:4em; color:orange;">DATA PREPARATION</p>

<p style="font-size:3em; color:orange;">NUMERICAS</p>
<p style="font-size:2em; color:orange;">tratamiento de los NULOS</p>

In [ ]:
# creo una lista con las columnas NUMERICAS que tienen NULOS (35 sobre 52)
lista_numericas_con_NULOS = df[lista_columnas_numericas].columns[df[lista_columnas_numericas].isna().any()].tolist()

In [ ]:
# asigno -3 a cada valor NaN de las columnas NUMERICAS
mf.rellena_con_valor(df, lista_numericas_con_NULOS, -3)
# compruebo si quedan NaN en la columnas NUMERICAS
print("compruebo que no quedan nulos en las numericas: " + str(df[lista_numericas_con_NULOS].isnull().any().sum()))

<p style="font-size:2em; color:orange;">corelacion con el target</p>

In [ ]:
# corelacion de la variable con el target
plt.figure(figsize=[17,3])
plt.xticks(rotation=90)
sns.countplot(data=df, x='Wdft_RegionIdentifier', hue='HasDetections')

# hay una corelacion mas animada del target con la siguientes variables:
# AVProductsInstalled
# CountryIdentifier
# Census_ProcessorCoreCount
# Census_OSBuildNumber
# Census_OSUILocaleIdentifier
# Census_IsTouchEnabled 
# Wdft_IsGamer 
# Wdft_RegionIdentifier 

<p style="font-size:3em; color:orange;">CATEGORICAS con NULOS</p>
<p style="font-size:1em; color:orange;">· tratamiento de los NULOS</p>

In [ ]:
# saco una lista con las columnas CATEGORICAS que tienen NULOS (7 sobre 28)
lista_categoricas_con_NULOS = df[lista_columnas_categoricas].columns[df[lista_columnas_categoricas].isna().any()].tolist()

In [ ]:
lista_categoricas_con_NULOS

<p style="font-size:1em; color:orange;">por todas las variables categoricas, junto en la variable "Other_xxxxxx" los NaN y los atributos con poca representacion y compruebo</p>
<p style="font-size:1em; color:orange;">utilizo de la funcion setOthers</p>

## OsBuildLab

In [ ]:
# me quedo con los 9 primeros atributos
df['OsBuildLab'] = mf.setOthers(df, 'OsBuildLab', 9, 'Other_OsBuildLab')

In [ ]:
# compruebo
df['OsBuildLab'].value_counts(dropna=False).head(30)

## SmartScreen

In [ ]:
# me quedo con los 6 primeros atributos
df['SmartScreen'] = mf.setOthers(df, 'SmartScreen', 6, 'Other_SmartScreen')

In [ ]:
# compruebo
df['SmartScreen'].value_counts(dropna=False)

## Census_ProcessorClass

In [ ]:
# me quedo con los 3 primeros atributos
df['Census_ProcessorClass'].value_counts(dropna=False)

In [ ]:
# compruebo
df['Census_ProcessorClass'] = mf.setOthers(df, 'Census_ProcessorClass', 3, 'Other_Census_ProcessorClass')

## Census_PrimaryDiskTypeName

In [ ]:
# me quedo con los 2 primeros atributos
df['Census_PrimaryDiskTypeName'] = mf.setOthers(df, 'Census_PrimaryDiskTypeName', 2, 'Other_Census_PrimaryDiskTypeName')

In [ ]:
# compruebo
df['Census_PrimaryDiskTypeName'].value_counts(dropna=False)

## Census_ChassisTypeName

In [ ]:
# me quedo con los 20 primeros atributos
df['Census_ChassisTypeName'] = mf.setOthers(df, 'Census_ChassisTypeName', 20, 'Other_Census_ChassisTypeName')

In [ ]:
# compruebo
df['Census_ChassisTypeName'].value_counts(dropna=False)

## Census_PowerPlatformRoleName

In [ ]:
# me quedo con los 7 primeros atributos
df['Census_PowerPlatformRoleName'] = mf.setOthers(df, 'Census_PowerPlatformRoleName', 7, 'Other_Census_PowerPlatformRoleName')

In [ ]:
# compruebo
df['Census_PowerPlatformRoleName'].value_counts(dropna=False)

## Census_InternalBatteryType

In [ ]:
# me quedo con los 11 primeros atributos
df['Census_InternalBatteryType'] = mf.setOthers(df, 'Census_InternalBatteryType', 11, 'Other_Census_InternalBatteryType')

In [ ]:
# compruebo
df['Census_InternalBatteryType'].value_counts(dropna=False)

<p style="font-size:1em; color:orange;">distribucion target/variable y pivot table</p>

In [ ]:
# miro la distribucion del target en relacion con la variable numerica
plt.figure(figsize=[17,3])
plt.xticks(rotation=90)
sns.countplot(data=df, x='OsBuildLab', hue='HasDetections')

In [ ]:
# con las variables CATEGORICAS, muy interesante es la pivot table 
# como se corelacione el target con la variable eligida

df.pivot_table(index='OsBuildLab', values='HasDetections', aggfunc=[len, sum, np.mean]).sort_values(by=[('mean','HasDetections')], ascending = False)

#  por el valor de la variable OsBuildLab fila1 tengo un total de 206436, 109335 son infectado, osea un 52,9%
# len (count): cuenta cuantas observaciones tengo para cada etiqueta del OsBuildLab
# sum (total de OsBuilLb infectacdo): suma el target para cada una de las etiquetas, suma los infectado por este valor de la etiqueta OsBuildLab
# mean(proporcion de infectados): me da el porcentaje de los infectados

<p style="font-size:3em; color:orange;">CATEGORICAS sin NULOS</p>
<p style="font-size:1em; color:orange;">· reduccion de los atrobutos</p>
<p style="font-size:1em; color:orange;">· reduccion de los menos representativos</p>

In [ ]:
# aqui miro por cada CATEGORICAS cuantos valores diferentes tiene la variable
for i in lista_columnas_categoricas:
    print(i)
    print(df[i].nunique())
    print('\n')

## EngineVersion

In [ ]:
# reduccion de etiqueta
df['EngineVersion'] = df['EngineVersion'].str[:7]

In [ ]:
# me quedo con los primero 25 atributos
df['EngineVersion'] = mf.setOthers(df, 'EngineVersion', 25, 'Other_EngineVersion')

## AppVersion

In [ ]:
df['AppVersion'] = df['AppVersion'].str[:4]

In [ ]:
df['AppVersion'] = mf.setOthers(df, 'AppVersion', 9, 'Other_AppVersion')

## AvSigVersion

In [ ]:
df['AvSigVersion'] = df['AvSigVersion'].str[:4]

In [ ]:
df['AvSigVersion'] = mf.setOthers(df, 'AvSigVersion', 6, 'Other_AvSigVersion')

## OsVer

In [ ]:
df['OsVer'] = mf.setOthers(df, 'OsVer', 3, 'Other_OsVer')

In [ ]:
df['OsVer'].value_counts().count()

## Census_OSVersion

In [ ]:
df['Census_OSVersion'] = df['Census_OSVersion'].str[:7]

In [ ]:
df['Census_OSVersion'] = mf.setOthers(df, 'Census_OSVersion', 5, 'Other_Census_OSVersion')

## Census_OSBranch

In [ ]:
df['Census_OSBranch'] = mf.setOthers(df, 'Census_OSBranch', 12, 'Other_Census_OSBranch')

## Census_OSEdition

In [ ]:
df['Census_OSEdition'] = mf.setOthers(df, 'Census_OSEdition', 13, 'Other_Census_OSEdition')

## Census_OSSkuName

In [ ]:
df['Census_OSSkuName'] = mf.setOthers(df, 'Census_OSSkuName', 11, 'Other_Census_OSSkuName')

## Census_FlightRing

In [ ]:
df['Census_FlightRing'] = mf.setOthers(df, 'Census_FlightRing', 6, 'Other_Census_FlightRing')

<p style="font-size:3em; color:orange;">CATEGORICAS todas</p>
<p style="font-size:1em; color:orange;">OHE</p>

In [ ]:
len(lista_columnas_categoricas)

In [ ]:
for elementos in lista_columnas_categoricas:
    df = mf.OHE(df,elementos)

In [ ]:
df.info() # tengo ahora 285 columnas

<p style="font-size:4em; color:DodgerBlue;">MODELING & EVALUATION</p>

<p style="font-size:3em; color:DodgerBlue;">PARTICION del DATAFRAME</p>
<p style="font-size:1em; color:DodgerBlue;">DEVELOPMENT y VALIDATION (aleatorio con porcentaje)</p>

In [ ]:
# hago un subset de mi DF
#development_df = df[df['Platform_windows10']==0].sample(frac=0.87,random_state=42) # development = train + test
#validation_df = df.drop(development_df.index) # validation

validation_df = df[df['Platform_windows10'] == 1].sample(frac=0.2)
development_df = pd.merge(df,validation_df, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)

In [ ]:
development_df.shape

In [ ]:
validation_df.shape

In [ ]:
# DEVELOPMENT necesita los atributos y target por separados
dev_df_x = development_df.drop('HasDetections', axis=1) # atributos (sin inplace=True - no me cargo mi target) quiero solo un dev sin target
dev_df_y = development_df[['HasDetections']] # target

In [ ]:
# VALIDATION necesita los atributos y target por separados
val_df_x = validation_df.drop('HasDetections', axis=1) # atributos (sin inplace=True - no me cargo mi target) quiero solo un dev sin target
val_df_y = validation_df[['HasDetections']] # target

<p style="font-size:3em; color:DodgerBlue;">PARTICION del DEVELOPMENT</p>
<p style="font-size:1em; color:DodgerBlue;">TRAIN y TEST</p>

In [ ]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(
                                        dev_df_x, # X 
                                        dev_df_y, # y
                                        test_size = 0.20, # tamaño del split aleatorio 
                                        random_state = 42
                                     )

<p style="font-size:3em; color:DodgerBlue;">FITEAR el MODELO</p>
<p style="font-size:1em; color:DodgerBlue;">Instanciar el modelo: escoger el algoritmo y definir los hiperparámetros</p>
<p style="font-size:1em; color:DodgerBlue;">Entrenar el modelo: el algoritmo aprende las reglas concretas a partir de los datos train</p>
<p style="font-size:1em; color:DodgerBlue;">Predecir: calcular la probabilidad de infeccion sobre un dataset nuevo -> test -> que nos permita medir la bonanza del modelo</p>
<p style="font-size:1em; color:DodgerBlue;">Evaluar: compara las  predicciones que tengo en test con valor real del test</p>

<p style="font-size:2em; color:DodgerBlue;">INSTANCIAR EL MODELO</p>

In [ ]:
# INSTANCIAR el modelo
dt = DecisionTreeClassifier(
                        max_depth=20, # Importante, numero de preguntas que realiza el algoritmo
                        min_samples_leaf=200, # solo para el nodo terminal, numero de observaciones minima
                        random_state=42,
                )

<p style="font-size:2em; color:DodgerBlue;">ENTRENAR EL MODELO</p>

In [ ]:
# ENTRENAR el modelo
dt.fit(x_train, y_train)

In [ ]:
# revisamos las reglas que ha generado el modelo
dot_data = export_graphviz(decision_tree= dt,
                out_file=None,
                feature_names= x_test.columns,
                class_names= ['not infected','infected']
)

In [ ]:
graphviz.Source(dot_data)

<p style="font-size:2em; color:DodgerBlue;">PREDICCION DEL MODELO</p>

In [ ]:
# PREDICCION DEL MODELO
dt.predict(x_test)
#  me dice la prediccion del modelo

In [ ]:
#dt.predict_proba(x_test)
# doble prediccion: probabilidad de no infectado y de infectado

In [ ]:
# aqui junto en un dataframe la predicciones del modelo
y_test_pred = pd.DataFrame(dt.predict(x_test), index= y_test.index, columns= ['DetectionPrediction'])

In [ ]:
# pongo uno a lado del otro el test y la prediccion
results_df = y_test.join(y_test_pred)

In [ ]:
# comparo la prediccion con lo que tiene realmente el target
results_df['PREDICCION_LOGRADA'] = (results_df['DetectionPrediction'] == results_df['HasDetections']).astype(int)

In [ ]:
results_df.head()

<p style="font-size:2em; color:DodgerBlue;">EVALUACION DEL MODELO</p>

<p style="font-size:2em; color:DodgerBlue;">ACCURACY</p>

In [ ]:
# aqui tengo un accuracy (el porcentaje de acierto) de mas o menos 60%
results_df['PREDICCION_LOGRADA'].mean()

In [ ]:
# forma alternativa de medir el accuracy
dt.score(x_test,y_test)


<p style="font-size:2em; color:DodgerBlue;background-color: white">conclusion:</p>
<p style="font-size:1em; color:DodgerBlue;">TASA DE APRENDIZAJE de 50%</p>
<p style="font-size:1em; color:DodgerBlue;">ACCURACY de 60%</p>
<p style="font-size:1em; color:DodgerBlue;">tengo un modelo que poco mas de la midad de las veces (60%) va a predecir corecto y un 40% no va a predecir corecto</p>

<p style="font-size:2em; color:DodgerBlue;">PRECISION y RECALL</p>

In [ ]:
# genero la confusion matrix
confusion_matrix = pd.crosstab(results_df['HasDetections'], results_df['DetectionPrediction'])

In [ ]:
confusion_matrix
#TN = 19454 no infectado que predigo y realmente no infectado
#TP = 29443 infectado que predigo y realmente son infectado
#FN = 11031 predigo negativo y es positivo
#FP = 20738 predigo positivo sin que lo sea

In [ ]:
TP = confusion_matrix.iloc[1,1]
TN = confusion_matrix.iloc[0,0]
FP = confusion_matrix.iloc[0,1]
FN = confusion_matrix.iloc[1,0]

In [ ]:
precision = TP / (TP+FP)
recall = TP / (TP + FN)

In [ ]:
precision # de todos los infectado que predigo, un 63% si es infectado

In [ ]:
recall # de todos los infectado que hay en el dataset predigo un 72%

<p style="font-size:2em; color:DodgerBlue;background-color: white">conclusion:</p>
<p style="font-size:1em; color:DodgerBlue;">de todos los infectado que predigo, un 63% si es infectado (precision)</p>
<p style="font-size:1em; color:DodgerBlue;">de todos los infectado que hay en el dataset predigo un 61% (recall)</p>

In [ ]:
# EVALUACION DEL MODELO CON DIFERENTES HIPERPARAMETROS

for i in range(1,30):
    dt2 = DecisionTreeClassifier(max_depth=i, min_samples_leaf=200,random_state=42)
    dt2.fit(x_train, y_train)
    train_accuracy = dt2.score(x_train, y_train)
    test_accuracy = dt2.score(x_test,y_test)
    print('max_depth del arbol {} ---> accuracy en train: {} ---> accuracy en test: {}' .format(i,train_accuracy,test_accuracy))

In [ ]:
# el max_depth optimal es de 20
# mas alla de 20, generamos OVERFITTING

In [ ]:
# como funciona el modelo en VALIDACION ?
dt_final = DecisionTreeClassifier(max_depth=20, min_samples_leaf=200,random_state=42)
dt_final.fit(x_train, y_train)
train_accuracy = dt_final.score(x_train, y_train)
test_accuracy = dt_final.score(x_test,y_test)
val_accuracy = dt_final.score(val_df_x,val_df_y)
print("ACCURACY en train: {} ACCURACY en test: {} ACCURACY en validation: {}".format(train_accuracy, test_accuracy,val_accuracy))

<p style="font-size:2em; color:DodgerBlue; background-color: white">conclusion:</p>
<p style="font-size:1em; color:DodgerBlue;">en train tengo un accuracy del 64%</p>
<p style="font-size:1em; color:DodgerBlue;">en test tengo un accuracy del 62%</p>
<p style="font-size:1em; color:DodgerBlue;">tambien en VALIDACION tengo una buena accuracy del 62%</p>